
Originally created by Chelsey McGowan-Yallop, SAMS-UHI (sa06cm@sams.ac.uk)

Modified by Muriel Dunn for AFKABAN then AZKABAN

This script uses model-predicted TS(f) spectra to train a machine learning
classifier, performs nested cross-validation, applies the classifier to
measured TS(f) spectra and outputs results files.

To use a different classifier, see the list of supported classifiers at:
https://github.com/hyperopt/hyperopt-sklearn and set as clf.

Sometimes the initial hyperparameter configuration selected by HyperOpt in each
split in the outer loop will be unsuccessful and all trials will fail. The
retry decorator forces it to try again until retry_limit is reached.

OUTPUT FILES:
    _NestedCV.pkl contains results of nested cross-validation procedure
    _Predictions.pkl contains measured TS(f) spectra with predicted labels
    _BestParams.pkl contains the optimal hyperparameters for the model
"""

In [1]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_classifier, svc, lightgbm_classification, gaussian_nb, any_preprocessing,min_max_scaler, normalizer
import lightgbm
from datetime import timedelta, date
from tenacity import retry, stop_after_attempt
import AZKABANML
import sys, errno  


# Read the dataframes

In [2]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path_df = 'C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/AZKABAN-light/ZoopMix_paper/'

# Mesocosm measurements
df_SED = pd.read_pickle(path_df+'df_SED.pkl')
df_track = pd.read_pickle(path_df+'df_track.pkl')
df_trackavg = pd.read_pickle(path_df+'df_trackavg.pkl')

# Labelled training set (scattering model results with smoothed shape file)
df_sim_smooth = pd.read_pickle(path_df+'df_sim_smooth.pkl')
df_sim_smooth_a = pd.read_pickle(path_df+'df_sim_smooth_antarctica.pkl')

### Partition modeled target spectra

In [3]:
### select bandwith 200
df_sim_200 = df_sim_smooth.loc[:,185.0:255.0].copy()
species_data = df_sim_smooth.loc[:,'spec']
df_sim_200.loc[:,'Species'] = species_data

# 70 kHz
df_sim_70 = df_sim_smooth.loc[:,45.0:90.0].copy()
df_sim_70.loc[:,'Species'] = species_data

# 120 kHz
df_sim_120 = df_sim_smooth.loc[:,70.0:190.0].copy()
df_sim_120.loc[:,'Species'] = species_data

# 333 kHz
df_sim_333 = df_sim_smooth.loc[:,283.0:383.0].copy()
df_sim_333.loc[:,'Species'] = species_data

# full kHz
df_sim_full = df_sim_smooth.loc[:,45.0:383.0].copy()
df_sim_full.loc[:,'Species'] = species_data

In [4]:
# Calculate linear
df_sim_200_sigbs = 10**(df_sim_smooth.loc[:,185.0:255.0].copy()/10)
species_data = df_sim_smooth.loc[:,'spec']
df_sim_200_sigbs.loc[:,'Species'] = species_data

# Run different iterations


## Preprocessing
Any! Give better results (+~0.2 on mean F1 score), either scale or min,max scaler or PCA.

In [5]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/Nya_AZKABAN/'
classifypath = 'F:/Nya-AZKABAN/Classify/'
d1 = date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
#preprocessing = [min_max_scaler(unique_id)] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

### kNN

In [5]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+d1+'_smooth' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath)

Resuming cross-validation from fold 11
Retraining model on full dataset
100%|████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.59s/trial, best loss: 0.3267385467222379]
Classifying new data


In [6]:
# CLASSIFIER kNN normalizer
unique_id = 'kNN_'+d1+'_smooth_norm' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 11
Retraining model on full dataset
100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.21s/trial, best loss: 0.3001387490032339]
Classifying new data


In [7]:
unique_id = 'lightGBM_'+d1+'_smooth_norm' #que ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 9
Beginning fold 9 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:03<00:00, 303.96s/trial, best loss=?]
Resuming cross-validation from fold 9
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 50/50 [03:51<00:00, 231.75s/trial, best loss: 0.2915634307272623]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:04<00:00, 304.54s/trial, best loss=?]
Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [01:02<00:00, 62.86s/trial, best loss: 0.28626383807344247]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 32/32 [05:04<00:00, 304.63s/trial, best loss: 0.28692327712021526]


100%|██████████████████████████████████████████████| 50/50 [02:40<00:00, 160.20s/trial, best loss: 0.28692327712021526]
Classifying new data


In [5]:
unique_id = 'SVC_'+d1+'_smooth_norm' #que ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_all(df_sim_200, df_SED, df_track, df_trackavg, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|██████████████████████████████████████████████| 50/50 [05:04<00:00, 304.40s/trial, best loss: 0.40975709057255716]
Retraining model on full dataset
100%|████████████████████████████████████████████████| 50/50 [00:17<00:00, 17.43s/trial, best loss: 0.4016067004086973]
Classifying new data


## Bandwidth runs

In [7]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_70' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_70, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.76s/trial, best loss: 0.2274688321828947]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:06<00:00,  6.25s/trial, best loss: 0.22836891921537106]


100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.46s/trial, best loss: 0.2232182216661288]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.43s/trial, best loss: 0.22616792309689626]


100%|███████████████████████████████████████████████| 50/50 [00:12<00:00, 12.34s/trial, best loss: 0.22616792309689626]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.99s/trial, best loss: 0.22937213690918956]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [00:05<00:00,  5.51s/trial, best loss: 0.23331179157940607]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.17s/trial, best loss: 0.22365390613683733]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:06<00:00,  6.22s/trial, best loss: 0.22404042788544198]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.71s/trial, best loss: 0.21957349106267643]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:06<00:00,  6.28s/trial, best loss: 0.22311766523743992]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.80s/trial, best loss: 0.22311766523743992]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.00s/trial, best loss: 0.22183125741846277]
Beginning fold 9 of 10
100%|██████████████████████████████████████████████████| 5/5 [00:07<00:00,  7.52s/trial, best loss: 0.2257262713044722]


100%|████████████████████████████████████████████████| 50/50 [00:08<00:00,  8.79s/trial, best loss: 0.2225895669174408]
Beginning fold 10 of 10
100%|████████████████████████████████████████████████| 23/23 [00:11<00:00, 11.59s/trial, best loss: 0.2244892184794296]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.22s/trial, best loss: 0.22291020650994264]
Retraining model on full dataset
100%|████████████████████████████████████████████████| 41/41 [00:06<00:00,  6.21s/trial, best loss: 0.2230834705051058]


100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.27s/trial, best loss: 0.2230834705051058]


In [8]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_120' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_120, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.71s/trial, best loss: 0.15461726658117136]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:19<00:00, 19.15s/trial, best loss: 0.15068822842259766]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.33s/trial, best loss: 0.14916123174748397]
Beginning fold 3 of 10
100%|████████████████████████████████████████████████| 35/35 [00:06<00:00,  6.47s/trial, best loss: 0.1543123578126021]


100%|████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.87s/trial, best loss: 0.1543123578126021]
Beginning fold 4 of 10
100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.84s/trial, best loss: 0.1493982923079038]
Beginning fold 5 of 10
100%|██████████████████████████████████████████████████| 2/2 [00:09<00:00,  9.41s/trial, best loss: 0.1529729322079023]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.91s/trial, best loss: 0.14677649058921194]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:06<00:00,  6.56s/trial, best loss: 0.15068597638002257]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.69s/trial, best loss: 0.14953706132085098]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:04<00:00,  4.68s/trial, best loss: 0.14893818856080632]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.78s/trial, best loss: 0.14893818856080632]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.30s/trial, best loss: 0.14737210738496864]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:05<00:00,  5.27s/trial, best loss: 0.14569630193029615]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.78s/trial, best loss: 0.14364066738900283]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 23/23 [00:07<00:00,  7.80s/trial, best loss: 0.14217726252436325]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.25s/trial, best loss: 0.14217726252436325]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:05<00:00,  5.88s/trial, best loss: 0.14783588079881493]


100%|███████████████████████████████████████████████| 50/50 [00:18<00:00, 18.92s/trial, best loss: 0.14783588079881493]


In [9]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_333' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_333, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.84s/trial, best loss: 0.36222047861968365]
Beginning fold 2 of 10
100%|████████████████████████████████████████████████| 17/17 [00:05<00:00,  5.01s/trial, best loss: 0.3586949427462398]


100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.45s/trial, best loss: 0.3586949427462398]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:18<00:00, 18.52s/trial, best loss: 0.35486341313967174]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.90s/trial, best loss: 0.35430617582212454]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:09<00:00,  9.33s/trial, best loss: 0.35920863458696217]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [00:06<00:00,  6.89s/trial, best loss: 0.38530562652437805]


100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.40s/trial, best loss: 0.3648927167159851]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:04<00:00,  4.55s/trial, best loss: 0.36324971800498485]


100%|███████████████████████████████████████████████| 50/50 [00:15<00:00, 15.47s/trial, best loss: 0.36197928344012487]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:06<00:00,  6.26s/trial, best loss: 0.36819072872057257]


100%|███████████████████████████████████████████████| 50/50 [00:19<00:00, 19.16s/trial, best loss: 0.36819072872057257]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.29s/trial, best loss: 0.3614867901782103]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:07<00:00,  7.19s/trial, best loss: 0.36124022487822816]


100%|███████████████████████████████████████████████| 50/50 [00:19<00:00, 19.26s/trial, best loss: 0.36124022487822816]
Beginning fold 10 of 10
100%|████████████████████████████████████████████████| 23/23 [00:05<00:00,  5.34s/trial, best loss: 0.3630731882554692]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.63s/trial, best loss: 0.36132217775331965]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:06<00:00,  6.37s/trial, best loss: 0.35492183137317734]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.38s/trial, best loss: 0.35492183137317734]


In [10]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_full' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_full, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:07<00:00,  7.67s/trial, best loss: 0.08310690780988528]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 17/17 [00:09<00:00,  9.43s/trial, best loss: 0.08141592683865662]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.33s/trial, best loss: 0.08141592683865662]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:08<00:00,  8.25s/trial, best loss: 0.08564760171629415]


100%|███████████████████████████████████████████████| 50/50 [00:07<00:00,  7.48s/trial, best loss: 0.08564760171629415]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:07<00:00,  7.63s/trial, best loss: 0.08104287487994699]
Beginning fold 5 of 10
100%|█████████████████████████████████████████████████| 2/2 [00:06<00:00,  6.50s/trial, best loss: 0.09479488475816611]


100%|████████████████████████████████████████████████| 50/50 [00:08<00:00,  8.23s/trial, best loss: 0.0789158916746927]
Beginning fold 6 of 10
100%|████████████████████████████████████████████████| 20/20 [00:08<00:00,  9.00s/trial, best loss: 0.0851028215940729]


100%|████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.25s/trial, best loss: 0.0851028215940729]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 38/38 [00:06<00:00,  6.99s/trial, best loss: 0.08605892827037775]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.70s/trial, best loss: 0.08605892827037775]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:07<00:00,  7.82s/trial, best loss: 0.08162459774940634]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:07<00:00,  7.41s/trial, best loss: 0.09418757247728304]


100%|████████████████████████████████████████████████| 50/50 [00:07<00:00,  7.97s/trial, best loss: 0.0855494016364653]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 23/23 [00:53<00:00, 53.89s/trial, best loss: 0.08532721541509813]


100%|███████████████████████████████████████████████| 50/50 [00:18<00:00, 18.87s/trial, best loss: 0.08532721541509813]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:07<00:00,  7.15s/trial, best loss: 0.07720590512601111]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.87s/trial, best loss: 0.07720590512601111]


## Linear
Balanced + PP

In [6]:
# CLASSIFIER kNN with bandwidth chuncks
unique_id = 'kNN_'+ d1 +'_linear' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_sim_200_sigbs, clf, unique_id, classifypath, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.90s/trial, best loss: 0.2844933200972506]
Beginning fold 2 of 10
100%|████████████████████████████████████████████████| 17/17 [00:05<00:00,  5.94s/trial, best loss: 0.2839601583682937]


100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.11s/trial, best loss: 0.2829457558320698]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.47s/trial, best loss: 0.29351034057953374]


100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.44s/trial, best loss: 0.2889092050624633]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.27s/trial, best loss: 0.28285509867047576]
Beginning fold 5 of 10
100%|██████████████████████████████████████████████████| 2/2 [00:06<00:00,  6.20s/trial, best loss: 0.3417659841433709]


100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.40s/trial, best loss: 0.2983253132797987]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:12<00:00, 12.53s/trial, best loss: 0.27511336074473147]


100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.16s/trial, best loss: 0.2742110211204478]
Beginning fold 7 of 10
100%|█████████████████████████████████████████████████| 38/38 [00:14<00:00, 14.88s/trial, best loss: 0.280982923039079]


100%|█████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.64s/trial, best loss: 0.280982923039079]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.35s/trial, best loss: 0.2762298878267234]
Beginning fold 9 of 10
100%|██████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.78s/trial, best loss: 0.3074195343539564]


100%|███████████████████████████████████████████████| 50/50 [00:07<00:00,  7.04s/trial, best loss: 0.28886345560266946]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 23/23 [00:04<00:00,  4.44s/trial, best loss: 0.30001511420917404]


100%|███████████████████████████████████████████████| 50/50 [00:13<00:00, 13.05s/trial, best loss: 0.28643370789087397]
Retraining model on full dataset
100%|████████████████████████████████████████████████| 41/41 [00:06<00:00,  6.24s/trial, best loss: 0.2747291825283482]


100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.11s/trial, best loss: 0.2747291825283482]
